In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = SparkSession \
    .builder \
    .getOrCreate()

sc = spark.sparkContext

Creating RDD

In [11]:
### Create some Dump data for testing
df= spark.range (1000000)
df = df.select(df.id,df.id*2,df.id*3)
df= df.union (df)
df= df.union (df)
df= df.union (df)
df= df.union (df)
df= df.union (df)

### Convert DataFrame to RDD.
RDD = df.rdd.map (lambda x : str (x[0]) + ',' + str (x[1]) + ',' + str (x[2]))
### Save the file at a HDFS Path
RDD.coalesce(1).saveAsTextFile ('/home/phillipefs/spark_dev/pyspark-end-to-end-developer/0 - PracticeFiles/test_data')

In [15]:
rdd = sc.textFile('/home/phillipefs/spark_dev/pyspark-end-to-end-developer/0 - PracticeFiles/test_data')
rdd.getNumPartitions()

21

In [16]:
rdd.count()

32000000

In [18]:
rdd1 =rdd.filter (lambda x : int(x.split (',')[0]) ==1)
rdd1.getNumPartitions()

21

In [21]:
rdd1.count()

32

coalesce()

In [22]:
rdd1 = rdd1.coalesce(1)
rdd1.getNumPartitions()

1

Repartition(numPartitions)
- Return a new RDD that has exactly numPartitions partitions
- Create almost equal sized partitions
- Can increase or decrease the level of parallelism
- Spark performs better with equal sized partitions If you need further processing of huge data, it is preferred to have a equal sized partitions
- and so we should consider using repartition
- Internally, this uses a shuffle to redistribute data from all partitions leading to very expensive operation So avoid if not required
- If you are decreasing the number of partitions, consider using coalesce where the movement of data shuffling across the partitions is lower

In [26]:
ord= sc.textFile ('/home/phillipefs/spark_dev/pyspark-end-to-end-developer/0 - PracticeFiles/Orders')
#Return num records partition. This RDD have 2 partitions.
ord.glom().map(len).collect()

[34564, 34319]

In [28]:
ord = ord.repartition(5)
ord.glom().map(len).collect()

[13770, 13780, 13774, 13780, 13779]

repartitionAndSortWithinPartitions(numPartitions =None, partitionFunc =<function portable_hash >, ascending=True, keyfunc)
- Repartition the RDD according to the given partitioner and, within each resulting partition, sort records by their keys.
- Need a key value paid RDD.

In [13]:
rdd= sc.parallelize (((9, ('a','z')), (3, ('x','f')), (6, ('j','b')),(4,('a','b')), (8,('s','b')), (1,('a','b'))),2)
rdd2 = rdd.repartitionAndSortWithinPartitions (2, lambda x : x % 2,True)
rdd2.glom().collect()

[[(4, ('a', 'b')), (6, ('j', 'b')), (8, ('s', 'b'))],
 [(1, ('a', 'b')), (3, ('x', 'f')), (9, ('a', 'z'))]]

coalesce(numPartitions , shuffle=False):
- Return a new RDD that is reduced into ` numPartitions `
- Optimized version of repartition().
- No shuffling.
- Results in a narrow dependency, e.g. if you go from 1000 partitions to 100 partitions, there will not be a shuffle,
instead each of the 100 new partitions will claim 10 of the current partitions.
- If a larger number of partitions is requested, it will stay at the current number of partitions.
- By Default Coalesce can be only used for decreasing the partitions. But by passing shuffle=True parameter it behaves
like repartition and we can increase the partitions as well.

In [17]:
rdd = sc.textFile('/home/phillipefs/spark_dev/pyspark-end-to-end-developer/0 - PracticeFiles/Orders',5)
rdd.glom().map(len).collect()

[13984, 13716, 13732, 13730, 13721]

In [18]:
rdd = rdd.coalesce(3)
rdd.glom().map(len).collect()

[13984, 27448, 27451]

In [19]:
rdd = rdd.coalesce(10, shuffle=True)
rdd.glom().map(len).collect()

[6870, 6880, 6890, 6900, 6900, 6900, 6900, 6889, 6880, 6874]